In [1]:
# Import libaries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
final_list = []

# Skytrax URL
url = 'https://www.airlinequality.com/airline-reviews/eva-air/?sortby=post_date%3ADesc&pagesize=50'

# Request URL
response = requests.get(url)

# Create Soup
soup = BeautifulSoup(response.content, 'lxml')

# For loop for HTML
for idx, review in enumerate(soup.find_all('div',{'class':"review-stats"})):
    
    # Create a empty dictionary to contain columns and values
    review_dict = dict()
    
    # Values that is in star fill
    names = review.find_all('td', class_='review-rating-header')[::-1]
    ratings = review.find_all('td', class_='stars')[::-1]
    
    for name, rating in zip(names[1:], ratings):
        review_dict[name.text] = rating.find_all(class_='fill')[-1].text
    
    # Author, Date, Overall_rating,Text_content, Recommended
    review_dict['Author'] = soup.find_all(itemprop='name')[idx+1].text
    review_dict['Date'] = soup.find_all(itemprop='datePublished')[::2][idx].attrs['content']
    review_dict['Overall_rating'] = soup.find_all('div',{'itemprop':'reviewRating'})[idx].find(itemprop="ratingValue").text
    review_dict['text_content'] = soup.find_all('div',class_='text_content')[idx].text
    review_dict['Recommended'] = soup.find_all('div',{'class':"review-stats"})[idx].find_all('tr')[-1].find_all('td')[-1].text
        
    # Type of Traveller
    try:
        review.find('td',{'class':'review-rating-header type_of_traveller'}).text is not None
        review_dict['traveller_type'] = soup.find_all('td',{'class':'review-rating-header type_of_traveller'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['traveller_type'] = np.nan
    
    # Seat type
    try:
        review.find('td',{'class':'review-rating-header cabin_flown'}).text is not None
        review_dict['seat_type'] = soup.find_all('td',{'class':'review-rating-header cabin_flown'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['seat_type'] = np.nan
    
    
    # Append dictionary into a list
    final_list.append(review_dict)

# Create DataFrame
df = pd.DataFrame(final_list)
df['airline'] = 'eva air'

# rearrange dataframe
df = df[['airline','Author','Date','text_content','traveller_type','seat_type','Overall_rating','Seat Comfort',
 'Cabin Staff Service','Food & Beverages','Inflight Entertainment','Ground Service','Value For Money',  
  'Recommended']]

# rename dataframe
df.columns = ['airline_name', 'author', 'date', 'content', 'type_traveller','cabin_flown', 'overall_rating', 'seat_comfort_rating',
'cabin_staff_rating', 'food_beverages_rating','inflight_entertainment_rating', 'ground_service_rating','value_money_rating', 'recommended']

# Save to CSV
df.to_csv('Data/eva_air.csv',index=False)

In [3]:
# Sanity check
df.head()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
0,eva air,N Partin,2020-05-07,"✅ Trip Verified | For weeks, I've been trying...",Business,Economy Class,1,NaN,NaN,NaN,NaN,NaN,1,no
1,eva air,W Harvey,2020-05-04,Not Verified | London to Bangkok . Very comfo...,Solo Leisure,Economy Class,9,4,4,4,4,3,4,yes
2,eva air,Mark Schneider,2020-04-10,✅ Trip Verified | Los Angeles to Bangkok via T...,Couple Leisure,Premium Economy,8,4,4,4,5,5,4,yes
3,eva air,Nick Santini,2020-04-04,✅ Trip Verified | On 03/22/2020 EVA Air cance...,Solo Leisure,Economy Class,1,NaN,NaN,NaN,NaN,NaN,1,no
4,eva air,Pierre Yves Becher,2020-03-30,✅ Trip Verified | Paris to Taipei. Clean and ...,Couple Leisure,Premium Economy,6,4,2,3,3,4,4,yes


In [4]:
df.tail()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
45,eva air,C Lan,2019-06-17,✅ Trip Verified | San Francisco to Singapore ...,Family Leisure,Economy Class,4,3,2,4,2,1,3,no
46,eva air,Nicky Lloyd,2019-06-10,✅ Trip Verified | Bangkok to London. Flight w...,Solo Leisure,Economy Class,1,2,1,1,NaN,2,2,no
47,eva air,Rosita Chua,2019-06-06,✅ Trip Verified | The flight was fine and I w...,Solo Leisure,Economy Class,5,3,4,1,3,4,3,no
48,eva air,S Heeny,2019-06-04,✅ Trip Verified | Bangkok to London on EVA Ai...,Business,Premium Economy,3,2,2,1,1,4,3,no
49,eva air,V Sanhaj,2019-06-02,✅ Trip Verified | Singapore to Vancouver via ...,Solo Leisure,Business Class,10,4,5,5,5,5,5,yes
